In [1]:
from lxml import etree
import json
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline  
import seaborn as sns
import regex as reg
import unicodedata

In [2]:
import numpy as np
import regex #

# Parsing the metadata:

This notebook contains a more annotated indepth explination on the code implememted in the parsing script:

The following function extracts the metadata provided in the xml files. We iteratively parse the file as it does not follow standard xml format perfectly, hence using a regular parser would lead to errors being thrown.  The reason the xml can not be parsed with a regular paser is that the following tag is not present: `xmlns:dc ="http://purl.org/dc/elements/1.1/"` . This is not an issue in the web version. 

The information we want to extract is in the sections of the qualifier, element and subject attributes.

In [3]:
def parse(file_path): 
    i = 0 
    tmp = ''
    key = ''
    xml2 = etree.iterparse(file_path, recover=True)
    data = []
    for action, elem in xml2:
        data.append((elem.attrib, elem.tag, elem.text))
    data_dict = {}
    
    
    for attrib, tag, text in data: 
        try : 
            tmp = key
            
            key = attrib.get('qualifier')
            element = attrib.get('element')
            
            #way to distinguish eliminate some nan!
            if key == 'none':
                key = element
            
            if key in data_dict.keys() : 
                i = i + 1 
                data_dict[key + str(i)] = text
            else : 
                i = 0 
                data_dict[key] = text
                
        except TypeError: 
            if 'subject' in tag:
                if 'subject' in data_dict.keys():
                    data_dict['subject'].append(text)
                else:
                    data_dict['subject'] = [text]
                    
        #parse irregular files
        if 'subject' not in data_dict.keys():
            f = open(file_path)
            text = f.read()
            data_dict['subject'] = reg.findall('&gt;([\w\-\ \;]*)&lt;', text)
            if len(data_dict['subject']) == 1:
                data_dict['subject'] = reg.split(';', data_dict['subject'][0])
            
    return data_dict

### Get all data 

We go over all the folders and parse all the xml files

In [4]:
import os
rootdir = '../papers-import/'

In [5]:
all_data = {}
i= 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        path = os.path.join(subdir, file)
        if 'dublin_core' in (path) :
            i += 1
            num_doc = subdir[len(rootdir):]
            all_data[num_doc] = parse(path)

In [6]:
print('got data from {} files'.format(i))

got data from 503 files


In [7]:
#add all the xml data into one dataframe
df_data = pd.DataFrame(list(all_data.values()), index=all_data.keys())

Convert to datetime format:

In [8]:
def convert(arg): 
    """Convert string to date time format"""
    try : 
        arg = dateutil.parser.parse(arg)
    except TypeError: 
        arg = arg 
    return arg 

In [9]:
df_data

,abstract,author3,available,type,None,issued,subject,uri,title,publisher,...,author15,author12,author13,author21,author22,author20,author25,author24,author26,author23
2018/import/234,Out-of-school labs (OSLs) aim at fostering stu...,"Brune, Laura",2018-11-04T23:26:23Z,Book chapter,authentic learning,2018-07,"[out-of-school learning, perceived authenticit...",https://doi.dx.org/10.22318/cscl2018.1061,Authentic Learning and Teaching in an Out-of-S...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/29,This study examined whether and how epistemic ...,"Tissenbaum, Catherine Louise Dornfeld",2018-11-04T23:14:21Z,Book chapter,epistemic cognition,2018-07,"[epistemic reflection, digital text, science l...",https://doi.dx.org/10.22318/cscl2018.240,Examining the Role of Explicit Epistemic Refle...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/445,It has been widely reported that youth are mor...,"Young, Rose Nicole",2018-11-04T23:35:55Z,Book chapter,transgender,2018-07,"[STEM, culture, intersectionality]",https://doi.dx.org/10.22318/cscl2018.1665,LGBT+ in STEM: The Transgender Experience,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/460,This paper focuses on the design and effects o...,NaN,2018-11-04T23:35:55Z,Book chapter,CSCL,2018-07,"[Civic Education, Epistemic Aims, Epistemic Co...",https://doi.dx.org/10.22318/cscl2018.1695,Exploring the Impact of Virtual Internships fo...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/417,Analyzing video data is a complex problem for ...,"Silver, Cindy Hmelo",2018-11-04T23:35:55Z,Book chapter,visualization,2018-07,"[video analysis, engineering design, human-cen...",https://doi.dx.org/10.22318/cscl2018.1609,Visualizing Complex Classrooms Through Real Ti...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/395,None,"Yung, Benny",2018-11-04T23:35:55Z,Book chapter,Video,2018-07,"[Nature of science, NOS-specific pedagogy, ini...",https://doi.dx.org/10.22318/cscl2018.1565,Using a video-based approach to develop pre-se...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/200,This paper describes the design of non-program...,"Basu, Satabdi",2018-11-04T23:26:23Z,Book chapter,computer science education,2018-07,"[introductory programming, deeper learning, de...",https://doi.dx.org/10.22318/cscl2018.925,Combining Non-Programming Activities with Prog...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/59,Abstract: Although efforts to critique deficit...,NaN,2018-11-04T23:14:21Z,Book chapter,equity,2018-07,"[dignity, justice, heterogeneity]",https://doi.dx.org/10.22318/cscl2018.480,Equity in the Learning Sciences: Recent Themes...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/345,Science assessments should evaluate the full c...,NaN,2018-11-04T23:35:55Z,Book chapter,science inquiry assessment,2018-07,"[educational data mining, natural language pro...",https://doi.dx.org/10.22318/cscl2018.1465,Unpacking Why Student Writing Does Not Match T...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018/import/491,The idea emergence is a critical nature of col...,NaN,2018-11-04T23:35:55Z,Book chapter,idea emergence,2018-07,"[collective knowledge advancement, socio-seman...",https://doi.dx.org/10.22318/cscl2018.1757,An Analysis of Collective Knowledge Advancemen...,International Society of the Learning Science...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import dateutil.parser
df_data['available'] = df_data['available'].apply(lambda x : convert(x))
df_data['accessioned'] = df_data['accessioned'].apply(lambda x : convert(x))

List of columns in the dataframe. We have one paper per row, with authors in the author columns:

In [11]:
df_data.columns

Index([   'abstract',     'author3',   'available',        'type',
                None,      'issued',     'subject',         'uri',
             'title',   'publisher',     'author1',      'author',
               'iso',    'citation', 'accessioned',     'author2',
           'author4',     'author5',     'author6',     'author7',
          'author10',     'author8',    'author14',    'author11',
          'author17',    'author18',     'author9',    'author16',
          'author19',    'author15',    'author12',    'author13',
          'author21',    'author22',    'author20',    'author25',
          'author24',    'author26',    'author23'],
      dtype='object')

# Getting all authors in to one dataframe:

Now we will change the above dataframe structure so that we only have one author column:

In [12]:
#go from multiple author columns to a single one
cleaning = df_data.reset_index().melt(
    id_vars=['index','subject', 'iso', 'uri','type','publisher','title', 
             'issued', 'accessioned', 'citation', 'available', 'abstract'])
cleaning = cleaning[cleaning.value.notna()]
cleaning = cleaning[cleaning.variable.notna()]

#remove garbage entries
cleaning = cleaning[cleaning.value.map(lambda x: len(x) > 2)]
#remove na
cleaning = cleaning[cleaning.value.notna()]

Associate an order to the authors: this will later be used to merge with the exracted emails:

In [13]:
import re
cleaning['author_order'] = cleaning.variable.map(
    lambda x: 0 if len(re.search('\d*$', x).group(0)) == 0 else int(re.search('\d*$', x).group(0)))

del cleaning['variable']

The dataframe currently looks like this:

In [18]:
cleaning.sort_values(by='index').head()

,index,subject,iso,uri,type,publisher,title,issued,accessioned,citation,available,abstract,value,author_order
1692,2015/import/105,"[music, intuitions, social negotiation]",en,https://doi.dx.org/10.22318/cscl2015.105,Book chapter,International Society of the Learning Science...,“That’s What Everyone Else Is Saying…”: Collab...,2015-07,2017-06-19 14:15:51+00:00,"Downton, M. P. (2015). “That’s What Everyone E...",2017-06-19 14:15:51+00:00,With the progression of more user-friendly mus...,"Downton, Michael P.",0
2075,2015/import/122,"[orchestration, discourse, inquiry, collaborat...",en,https://doi.dx.org/10.22318/cscl2015.122,Book chapter,International Society of the Learning Science...,The 3R Orchestration Cycle: Fostering Multi-Mo...,2015-07,2017-06-19 14:15:51+00:00,"Fong, C., Cober, R., Messina, R., Moher, T., M...",2017-06-19 14:15:51+00:00,This paper reports on second and third iterati...,"Messina, Richard",2
3081,2015/import/122,"[orchestration, discourse, inquiry, collaborat...",en,https://doi.dx.org/10.22318/cscl2015.122,Book chapter,International Society of the Learning Science...,The 3R Orchestration Cycle: Fostering Multi-Mo...,2015-07,2017-06-19 14:15:51+00:00,"Fong, C., Cober, R., Messina, R., Moher, T., M...",2017-06-19 14:15:51+00:00,This paper reports on second and third iterati...,"Peebles, Ben",5
3584,2015/import/122,"[orchestration, discourse, inquiry, collaborat...",en,https://doi.dx.org/10.22318/cscl2015.122,Book chapter,International Society of the Learning Science...,The 3R Orchestration Cycle: Fostering Multi-Mo...,2015-07,2017-06-19 14:15:51+00:00,"Fong, C., Cober, R., Messina, R., Moher, T., M...",2017-06-19 14:15:51+00:00,This paper reports on second and third iterati...,"Slotta, James D.",6
1572,2015/import/122,"[orchestration, discourse, inquiry, collaborat...",en,https://doi.dx.org/10.22318/cscl2015.122,Book chapter,International Society of the Learning Science...,The 3R Orchestration Cycle: Fostering Multi-Mo...,2015-07,2017-06-19 14:15:51+00:00,"Fong, C., Cober, R., Messina, R., Moher, T., M...",2017-06-19 14:15:51+00:00,This paper reports on second and third iterati...,"Fong, Cresencia",0


In [19]:
cleaning[cleaning.subject.map(len) == 0]['index'].unique()

array(['2015/import/334', '2018/import/292', '2018/import/267',
       '2018/import/244', '2018/import/249', '2018/import/251',
       '2018/import/454', '2018/import/167', '2018/import/348',
       '2015/import/262', '2015/import/172', '2018/import/525',
       '2018/import/410', '2018/import/465', '2018/import/236',
       '2018/import/253', '2018/import/480', '2018/import/529',
       '2018/import/427', '2018/import/515', '2018/import/516',
       '2018/import/370', '2018/import/521', '2018/import/523',
       '2018/import/520', '2018/import/289', '2018/import/527',
       '2018/import/528', '2018/import/259', '2018/import/519',
       '2018/import/518', '2018/import/517', '2018/import/513',
       '2018/import/512', '2018/import/522', '2018/import/524',
       '2018/import/511', '2018/import/514', '2018/import/526'],
      dtype=object)

In [20]:
len(cleaning[cleaning.subject.map(len) == 0]['index'].unique())

39

## Now parse citation to get the shortened name (which can be matched to refrences)

the `get_names` variable matches the general format of surnames in citations
the `section_before_year` identifies parts with contain the names in APA style citation, by looking for the section before the (year) which comes right after the names of the authors in this citation style

In [21]:
get_names = r'([\w\-\&]*[\,] [\p{L}\.\ ]+[\&\,]?)'
section_before_year = r'[\S\s]*\(\d{4}\)'

cleaning.reset_index(drop=True, inplace=True)

cleaning['shortend_names'] = cleaning.citation.map(lambda x: re.match(section_before_year, x, re.U).group(0)).map(
    lambda x: [x.replace(',', '').replace('&', '').rstrip() for x in regex.findall(get_names, x)])

cleaning['shortend_names'] = cleaning.apply(lambda x: x['shortend_names'][x['author_order']], axis=1)

Do some renaming of the columns to make them easier to understand:

In [24]:
cleaning.rename(columns={'index': 'file', 'value':'long_name'}, inplace=True)

In [25]:
#make the file name match the convention
cleaning['file'] = cleaning.file.map(lambda x: x.replace('/', '_'))

Extracting the identifier string, which can be used to match references in paper to papers in the dataset:

In [26]:
def get_authors_month(sentence, debug = False):
    """Gets name of authors + reference to time of publication"""
    sentence = unicodedata.normalize('NFC', sentence)
    #regex to match the pattern presented, people are misspelling prone, hence the long regex
    regex = r'[\p{L}\,\ \.\:\;\/\&\-\'\`\(\)\’\–\¨\…\‐\*\´\＆\\]*\([\,\ \p{L}\d\-]*(18|19|20)\d{2}[\,\ \p{L}\d\-]*\)'
    match_bad_year = r'[\S\s]*\((18|19|20)\d{2}\/(18|19|20)\d{2}\)'

    #we don't just have years, we also have sentences such as these instead of dates
    match_press = r'[\S\s]*\((i|I)n (P|p)ress|manuscript under review\)'
    match_forth = r'[\S\s]*\((f|F)orthcoming\)'
    match_accepted = r'[\S\s]*\((a|A)ccepted\)'
    match_submitted = r'[\S\s]*\((s|S)ubmitted\)'
    match_underreview = r'[\S\s]*\((u|U)nder (R|r)eview\)'

    #sentence = sentence.lower()
    if reg.match(regex, sentence):
        s = reg.search(regex, sentence).group(0)
        if len(s) > 9:
            return s
    elif re.match(match_bad_year, sentence):
        return re.search(match_bad_year, sentence).group(0)
    elif re.match(match_press, sentence):
        return re.search(match_press, sentence).group(0)
    elif re.match(match_forth, sentence):
        return re.search(match_forth, sentence).group(0)
    elif re.match(match_accepted, sentence):
        return re.search(match_accepted, sentence).group(0)
    elif re.match(match_submitted, sentence):
        return re.search(match_submitted, sentence).group(0)
    elif re.match(match_underreview, sentence):
        return re.search(match_underreview, sentence).group(0)

    return np.nan


def author_title(x):
    """Gets author and tite part of reference string"""
    ref = x
    authors = get_authors_month(x)  
    if isinstance(authors, float):
        return None
    
    search = len(authors)+1
    #In Looi is a comming occurence hence it is inlcuded here
    end = re.search('\.|\?|In Looi', ref[search:])
    if end:
        end = end.start()
    else:
        end = 0
    return ref[: (search+end)]

In [27]:
cleaning['identifier'] = cleaning[cleaning.citation.notna()].citation.map(lambda x: author_title(x))

## Unifying the names:


We see that often people spell their name differently across years, hence we try to unify the naming:

In [28]:
cleaning.long_name.sort_values().drop_duplicates().tail()

1225            van Aalst, Jan
1731    van Amersfoort, Daniël
1171           van Gog, Tamara
1191        van Heuven, Walter
866     van Leeuwen, Anouschka
Name: long_name, dtype: object

get the unique names and then use intersection to determine if one names is equal to another (we consider it equal if it contains at least two names that are equal)

In [29]:
names = cleaning.long_name.unique().tolist()

We "normalize" the names, that is convert them all into the same type of unicode formating to get even more overlap, otherwise, letters that look the same may not be matched by string comparison, as their unicode is different.

We visualy check that this way works by printing out matched. We can see that quite a lot of people have alternate versions of name spelling.

We will hoever keep some version of these alternate spellings to have a database of person names which will later be used when we extract affiliations from the papers themselves. Check the parsing universities notebook for more information. (Implemented in the scripts)

Reason why we normalize the string:

In [30]:
print(repr(u'ä'), ord(u'ä'))

for c in u"ä":
    print(repr(c), ord(c))

'ä' 228
'a' 97
'̈' 776


In [31]:
unicodedata.normalize('NFC', u'ä')

'ä'

In [32]:
d= {}
for i, m in enumerate(names):
    for j, n in enumerate(names):
        if i < j:
            y = set([i.lower() for i in reg.split(' |\,', unicodedata.normalize('NFC', m)) if len(reg.sub('\.', '', i)) > 2])
            name = set([i.lower() for i in reg.split(' |\,', unicodedata.normalize('NFC', n)) if len(reg.sub('\.', '', i)) > 2])
            if len(name.intersection(y)) > 1 and n!= m and not ('Lee' in n or 'Lee' in m):
                d[n]= m
                print(n, '|', m)


Dornfeld, Catherine | Tissenbaum, Catherine Louise Dornfeld
Krämer, Nicole | Krämer, Nicole
Rosé, Carolyn Penstein | Rosé, Carolyn
Margulieux, Lauren E. | Margulieux, Lauren E
Chan, Carol K.K. | Chan, Carol
Dyer, Elizabeth B. | Dyer, Elizabeth B
Levy, Sharona T. | Levy, Sharona
Williams, Joseph Jay | Williams, Joseph
Danish, Joshua A. | Danish, Joshua
Richard, Gabriela T. | Richard, Gabriela
Arastoopour, Golnaz | Irgens, Golnaz Arastoopour
Linn, Marcia | Linn, Marcia C.
Yoon, Susan A. | Yoon, Susan
Wise, Alyssa Friend | Wise, Alyssa
betser, sagit | Betser, Sagit
Keifert, Danielle | Keifert, Danielle Teodora
Kafai, Yasmin | kafai, yasmin
Flood, Virginia J. | Flood, Virginia J
Yip, Jason C. | Yip, Jason


We check the number of current names so we can see how the unification of spelling reduces the number of unique names.

In [33]:
len(cleaning.long_name.unique())

1306

We build a mapping using the above matched, and we can now us it to unify the naming:

In [34]:
cleaning.loc[cleaning['long_name'].isin(d.keys()), 'long_name'] = cleaning.long_name.map(d)

In [35]:
len(cleaning.long_name.unique())

1287

We can se that we reduce the number of distinct names quite a bit!

In [43]:
import networkx as nx

In [44]:
build_graph = cleaning[['long_name', 'shortend_names', 'file']]
build_graph = pd.merge(build_graph, build_graph, on='file')
build_graph = build_graph[build_graph.long_name_x != build_graph.long_name_y]

In [45]:
build_graph.head()

,long_name_x,shortend_names_x,file,long_name_y,shortend_names_y
1,"Brune, Laura",Brune L.,2018_import_234,"Keuschnig, Angelina",Keuschnig A.
2,"Brune, Laura",Brune L.,2018_import_234,"Nachtigall, Valentina",Nachtigall V.
3,"Brune, Laura",Brune L.,2018_import_234,"Behrendt, Lena",Behrendt L.
4,"Keuschnig, Angelina",Keuschnig A.,2018_import_234,"Brune, Laura",Brune L.
6,"Keuschnig, Angelina",Keuschnig A.,2018_import_234,"Nachtigall, Valentina",Nachtigall V.


In [46]:
G = nx.from_pandas_edgelist(build_graph, source='long_name_x', target='long_name_y')

In [47]:
import difflib

In [48]:
threshold = 0.8
d = dict()
for name in build_graph.long_name_x.unique():
    members = sorted(list(G.neighbors(name)))
    for i, member in enumerate(members):
            for j, member_2 in enumerate(members):
                if i < j:
                    difference = difflib.SequenceMatcher(None, member, member_2).ratio()
                    if difference > threshold:
                        print(member, '|' ,member_2)
                        d[member] = member_2

Sheldon, Josh | Sheldon, Joshua


In [49]:
d

{'Sheldon, Josh': 'Sheldon, Joshua'}

In [50]:
cleaning.loc[cleaning['long_name'].isin(d.keys()), 'long_name'] = cleaning.long_name.map(d)

In [51]:
cleaning.long_name.drop_duplicates().shape

(1286,)

Now that the dataset is clean we can explore it, check the analysis section for more information.